# Clustering

aca anotar lo q se quiere probar

recordar utilizar metricas de comparación

https://huggingface.co/julien-c/distilbert-feature-extraction

CountVectorizer, TFIDF, LSA, LDA, SBERT, spaCy

ideas:
    (1,2) ngrams of Qpos + Apos   
    (1,2) ngrams of Qlemmas
    (1,2) ngrams of Alemmas
    (1,2) ngrams of AQAQlemmas


ya tengo el codigo para agregar features


In [6]:
from TCA import *

import pandas as pd

df = pd.read_csv("db/features.csv", index_col=0)


,id,Q,Qscore,Qless,Qlemmas,Qpos,Qtoks,ANS,ANSscore,Aless,Alemmas,Apos,Atoks
4,100b8y,"First time drinking, what do I do?",7,"First time drinking, I ?","First time drinking , I ?","[('First', 'JJ'), ('time', 'NN'), ('drinking',...","['First', 'time', 'drinking', ',', 'I', '?']","try not mix drinks.., if you start with spirit...",12.0,"try mix drinks.., start spirits finish spirits..","try mix drink .. , start spirit finish spirit ..","[('try', 'VB'), ('mix', 'JJ'), ('drinks', 'NNS...","['try', 'mix', 'drinks', '..', ',', 'start', '..."
10,100nsq,Should I or should I not upgrade to Windows 8?,5,Should I I upgrade Windows 8?,Should I I upgrade Windows 8 ?,"[('Should', 'MD'), ('I', 'PRP'), ('I', 'PRP'),...","['Should', 'I', 'I', 'upgrade', 'Windows', '8'...",how about dualboot?,6.0,dualboot?,dualboot ?,"[('dualboot', 'NN'), ('?', '.')]","['dualboot', '?']"
38,102xlc,What makes you smile?,123,What makes smile?,What make smile ?,"[('What', 'WP'), ('makes', 'VBZ'), ('smile', '...","['What', 'makes', 'smile', '?']",zygomatic major and orbicularis oculi,172.0,zygomatic major orbicularis oculi,zygomatic major orbicularis oculus,"[('zygomatic', 'JJ'), ('major', 'JJ'), ('orbic...","['zygomatic', 'major', 'orbicularis', 'oculi']"
46,103c5q,What was your ego boost today?,5,What ego boost today?,What ego boost today ?,"[('What', 'WP'), ('ego', 'VBZ'), ('boost', 'NN...","['What', 'ego', 'boost', 'today', '?']",Yesterday I was told that I look like Emma Sto...,5.0,Yesterday I told I look like Emma Stone. I fee...,Yesterday I told I look like Emma Stone . I fe...,"[('Yesterday', 'NN'), ('I', 'PRP'), ('told', '...","['Yesterday', 'I', 'told', 'I', 'look', 'like'..."
47,103hv3,What is the worst way to die you can think of?,199,What worst way die think ?,What worst way die think ?,"[('What', 'WP'), ('worst', 'JJS'), ('way', 'NN...","['What', 'worst', 'way', 'die', 'think', '?']",there was a video from a few years ago of a ch...,201.0,video years ago chechnian guy getting throat c...,video year ago chechnian guy getting throat cu...,"[('video', 'CD'), ('years', 'NNS'), ('ago', 'R...","['video', 'years', 'ago', 'chechnian', 'guy', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
410527,ozgvlf,How did you get rid of depression?,1,How get rid depression?,How get rid depression ?,"[('How', 'WRB'), ('get', 'VB'), ('rid', 'JJ'),...","['How', 'get', 'rid', 'depression', '?']","I can't, I can only reduce my sadness by worki...",1.0,"I ', I reduce sadness working something I like","I ' , I reduce sadness working something I like","[('I', 'PRP'), (""'"", ""''""), (',', ','), ('I', ...","['I', ""'"", ',', 'I', 'reduce', 'sadness', 'wor..."
410528,ozgw90,What is your kink?,1,What kink?,What kink ?,"[('What', 'WP'), ('kink', 'VB'), ('?', '.')]","['What', 'kink', '?']",Snuff films.,1.0,Snuff films.,Snuff film .,"[('Snuff', 'NNP'), ('films', 'NNS'), ('.', '.')]","['Snuff', 'films', '.']"
410532,ozgwry,What time do you wake up and go to sleep?,1,What time wake go sleep?,What time wake go sleep ?,"[('What', 'WP'), ('time', 'NN'), ('wake', 'NN'...","['What', 'time', 'wake', 'go', 'sleep', '?']","Wake up in 5am, sleep in 8pm",1.0,"Wake 5am, sleep 8pm","Wake 5am , sleep 8pm","[('Wake', 'VB'), ('5am', 'CD'), (',', ','), ('...","['Wake', '5am', ',', 'sleep', '8pm']"
410534,ozgww8,What's the coolest thing about this generation?,1,What'coolest thing generation?,What'coolest thing generation ?,"[(""What'coolest"", 'JJS'), ('thing', 'NN'), ('g...","[""What'coolest"", 'thing', 'generation', '?']",Me and you\nYour momma and your cousin too,1.0,Me Your momma cousin,Me Your momma cousin,"[('Me', 'NNP'), ('Your', 'NNP'), ('momma', 'NN...","['Me', 'Your', 'momma', 'cousin']"


## Vectorial Representations

### CountVectorizer

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold

def cv_ngrams(docs, range=(2,3), min_df=20):
    
    #2 y 3 ngrams, minimo 20 (no se como elegir le numero correcto)
    cv = CountVectorizer(min_df=min_df, ngram_range=range)

    
    #Create, Normalize and Reduce 
    vspace = cv.fit_transform(docs)
    vspace = normalize(vspace, axis=1, norm="max")
    vspace = VarianceThreshold(threshold=1e-3).fit_transform(vspace)

    return vspace

vs = cv_ngrams(df["Q"])
vs


<410538x1372 sparse matrix of type '<class 'numpy.float64'>'
	with 2421023 stored elements in Compressed Sparse Row format>

### Embeddings


In [3]:
#First embedding
from sklearn.cluster import KMeans

kmc = KMeans(n_clusters=10, random_state=0)
kmc.fit(X=vs)

KMeans(n_clusters=10, random_state=0)

In [4]:
import random
q_clus = [[] for i in range(10)]

for sentence_id, cluster_id in enumerate(kmc.labels_):
    q_clus[cluster_id].append(df.iloc[sentence_id].Q)

for i, cluster in enumerate(q_clus):
    print("Cluster ", i+1)
    for tw in random.sample(cluster, 10):
        print(tw)

Cluster  1
What is the best American thing you've ever done while abroad?
If you were insanely angry on the verge of a mental breakdown what is the last song you would want to hear someone start blasting next to your ear?
What is the coolest invention by humans?
What is the scariest thing of being a doctor?
[Serious]What is the most creepiest/weirdest thing you've come across surfing the web?
Redditors that don't believe in spiritual or magical things, what is the most difficult thing to logically explain that you have experienced?
What is the most ridiculous customer complaint that you have received?
What is the: ""When in Rome...."" of your home town/city?
What is the ugliest animal or fish, outside of Scomo?
What is the biggest ongoing local drama happening in your neighborhood or workplace that is not important enough to be picked up by local or national news, but is interesting nonetheless?
Cluster  2
What are some Stupid little things that make you way too mad?
Need some new lo-f

In [47]:
#continuo viendo que funciona

#embeddings preentrenados
import gensim.downloader

#word2vec-google-news-300
w2v = gensim.downloader.load('glove-twitter-25')


[==================================================] 100.0% 104.8/104.8MB downloaded


In [59]:
w2v["mauro"]

array([-0.657   ,  0.27942 ,  0.6727  , -0.1725  , -0.91333 , -0.96729 ,
       -1.7812  ,  1.244   , -0.029866, -1.1127  ,  0.47211 , -0.87454 ,
       -0.92561 , -1.11    ,  1.1491  , -0.35654 , -0.60955 ,  0.20226 ,
       -0.13373 ,  0.56002 ,  0.25254 ,  0.57305 ,  0.14504 , -0.21456 ,
       -0.29104 ], dtype=float32)